In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!pip install praw
import praw

Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'praw'

In [ ]:
user_agent = "Ali 1.0 by /user/iallli"
reddit = praw.Reddit(
    client_id = 'JnvZiIF4j4Du62s36w_rjg', 
    client_secret = '0mPVZ3tcb2vbqCu94TEfw4byJieREw', 
    user_agent = user_agent
)

In [ ]:
import datetime

headlines = set()
for submission in reddit.subreddit('apple').top("all"):
    
    if (submission.selftext == ''):
        submission.selftext = 'This user did not comment on this.'
        
#     print('Author Name: '+str(submission.author))
#     print('Submission Title: '+submission.title)
#     print('Comment: '+submission.selftext)
#     print('Created Date: '+str(datetime.datetime.utcfromtimestamp(submission.created_utc)))
#     print('Score: '+str(submission.score))
#     print("\n")

    headlines.add(submission.title)
    #headlines.add(submission.selftext)
    
#print(len(headlines))
print(headlines)

In [ ]:
df = pd.DataFrame(headlines)
print(df)

In [ ]:
df.columns =['Headline']
print(df)

In [ ]:
df.to_csv('headlines.csv', header = False, encoding = 'utf-8', index = False)
df

In [1]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\SHAHEER\AppData\Roaming\nltk_data...


In [13]:
from textblob import TextBlob

def get_sentiment(text):
    # Create a TextBlob object from the input text
    blob = TextBlob(text)
    # Get the sentiment of the text
    sentiment = blob.sentiment.polarity
    # Return -1 for negative sentiment, 0 for neutral sentiment, and 1 for positive sentiment
    if sentiment < 0:
        return -1
    elif sentiment == 0:
        return 0
    else:
        return 1

# Test the function
print(get_sentiment("I will never use this product, because i don't like it!"))  # Output: 1
print(get_sentiment("I hate this product!"))  # Output: -1
print(get_sentiment("I'm neutral about this product."))  # Output: 0

0
-1
0


In [14]:
result = sia.polarity_scores("i will never use apple phone, because i don't like it")
# print(result)
if result['compound'] >= 0.05 :
        print("Positive")
elif result['compound'] <= - 0.05 :
    print("Negative")
else :
    print("Neutral")

Negative


In [ ]:
# import csv

# from textblob import TextBlob

# infile = 'headlines.csv'

# with open(infile, 'r') as csvfile:
#     rows = csv.reader(csvfile)
#     for row in rows:
#         sentence = row[0]
#         blob = TextBlob(sentence)
#         print(blob.sentiment)

In [ ]:
result = []
for line in headlines:
    
    pol_score = sia(line)
    pol_score['headlines'] = pol_score.sentimentline
    result.append(pol_score)
    
pprint(result[:5], width = 100)

In [ ]:
df = pd.DataFrame.from_records(result)
df.head()

In [ ]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

In [ ]:
df2 = df[['headlines', 'label']]

In [ ]:
df2.to_csv('reddit_headlines_label.csv', encoding = 'utf-8', index = False)

In [ ]:
df.label.value_counts()

In [ ]:
df.label.value_counts(normalize = True)*100

In [ ]:
print("Positive Headlines:\n")
pprint(list(df[df['label'] == 1].headlines)[:5], width = 1000)

print("\nNegative Headlines:\n")
pprint(list(df[df['label'] == -1].headlines)[:5], width = 1000)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
counts = df.label.value_counts(normalize = True)*100
sns.barplot(x = counts.index, y = counts, ax = ax)
ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel("Percentage")
plt.show()